In [0]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [0]:
# Let's just make sure the model is loaded

!ollama pull qwen2.5
MODEL = "qwen2.5"

In [0]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [0]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [0]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a summary of this website in markdown. \
If it includes news or announcements, then summarize these too.(only if they are present)\n\n"
    user_prompt += website.text
    return user_prompt

In [0]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [0]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(model=MODEL, messages=messages_for(website))
    return response['message']['content']

In [0]:
summarize("https://www.creative-tim.com/twcomponents/cheatsheet/")

In [0]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [0]:
display_summary('https://edwarddonner.com')